In [1]:
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
conf = SparkConf().setMaster("local[1]").setAppName("ex38")
sc = SparkContext(conf=conf)
spark = SparkSession.builder.appName("ex38").getOrCreate()

25/01/10 12:36:50 WARN Utils: Your hostname, paloma-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/01/10 12:36:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/10 12:36:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/10 12:36:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
schema = "sensorId STRING, date STRING, PM10 FLOAT"
data = spark.read.csv("/home/paloma/Downloads/jupyter_spark/ex38/input.csv", header=False, schema=schema)
data.show()

+--------+----------+----+
|sensorId|      date|PM10|
+--------+----------+----+
|      s1|2016-01-01|20.5|
|      s2|2016-01-01|30.1|
|      s1|2016-01-02|60.2|
|      s2|2016-01-02|20.4|
|      s1|2016-01-03|55.5|
|      s2|2016-01-03|52.5|
+--------+----------+----+



In [3]:
data = data.rdd # transforma em rdd
data.collect()

[Row(sensorId='s1', date='2016-01-01', PM10=20.5),
 Row(sensorId='s2', date='2016-01-01', PM10=30.100000381469727),
 Row(sensorId='s1', date='2016-01-02', PM10=60.20000076293945),
 Row(sensorId='s2', date='2016-01-02', PM10=20.399999618530273),
 Row(sensorId='s1', date='2016-01-03', PM10=55.5),
 Row(sensorId='s2', date='2016-01-03', PM10=52.5)]

In [4]:
thresh = 50
data = data.map(lambda line: (line.sensorId, line.date, round(line.PM10, 1))) # tuplas
data.collect()

[('s1', '2016-01-01', 20.5),
 ('s2', '2016-01-01', 30.1),
 ('s1', '2016-01-02', 60.2),
 ('s2', '2016-01-02', 20.4),
 ('s1', '2016-01-03', 55.5),
 ('s2', '2016-01-03', 52.5)]

In [5]:
sensor_thresh = data.filter(lambda line: line[2] > 50)
sensor_thresh.collect()

[('s1', '2016-01-02', 60.2),
 ('s1', '2016-01-03', 55.5),
 ('s2', '2016-01-03', 52.5)]

In [6]:
# retorna pares de tuplas com o id do sensor e "1", que simboliza uma ocorrencia do id
# esses pares funcionarao como pares (chave, valor)
counts = sensor_thresh.map(lambda line: (line[0], 1))
counts.collect()

[('s1', 1), ('s1', 1), ('s2', 1)]

In [ ]:
counts = counts.countByKey()
counts.collect()

In [ ]:
counts = counts.filter(lambda line: line[1] >= 2)
counts.collect()